In [92]:
import socket
import struct
import time
import random
import subprocess
import re
from scapy.all import Ether, ARP, srp, sendp, sniff
from scapy.all import *

To test our code, we'll need to create a virtual switch. TO do that, run `setup_script_mac.sh` if you're on mac. If you're not on mac, you brought this upon yourself.

Test sending a message to our bridge and receiving it back.

In [29]:
def send_frame(iface):
    # Create a simple Ethernet frame
    frame = Ether(dst="ff:ff:ff:ff:ff:ff", src=get_if_hwaddr(iface)) / Raw(load="Hello, virtual network!")
    sendp(frame, iface=iface, verbose=False)
    print(f"Sent frame on {iface}")

def receive_frame(iface):
    print(f"Listening on {iface}")
    sniff(iface=iface, prn=lambda x: x.show(), count=1)
    
def delayed_send(iface):
    time.sleep(5)
    send_frame(iface)


local_interface = "bridge0"
print(f"Using interface: {local_interface}")
# Start a thread to continuously send frames
send_thread = threading.Thread(target=delayed_send, args=(local_interface,))
send_thread.daemon = True
send_thread.start()
receive_frame(local_interface)

Using interface: bridge0
Listening on bridge0
###[ Ethernet ]### 
  dst       = ff:ff:ff:ff:ff:ff
  src       = 36:09:29:fa:8c:40
  type      = 0x9000
###[ Raw ]### 
     load      = 'Hello, virtual network!'
Sent frame on bridge0



Next, test the wifi bridge.

In [30]:
internet_interface = "bridge1"
print(f"Using interface: {internet_interface}")
for i in range(5):
    receive_frame(internet_interface)
# Visit a website/general internet traffic

Using interface: bridge1
Listening on bridge1
###[ 802.3 ]### 
  dst       = ff:ff:ff:ff:ff:ff
  src       = 38:f9:d3:95:b3:87
  len       = 46
###[ LLC ]### 
     dsap      = 0x0
     ssap      = 0x1
     ctrl      = 175
###[ Raw ]### 
        load      = '\\x81\x01\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

Listening on bridge1
###[ Ethernet ]### 
  dst       = ff:ff:ff:ff:ff:ff
  src       = 38:f9:d3:95:b3:87
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 328
     id        = 5967
     flags     = 
     frag      = 0
     ttl       = 255
     proto     = udp
     chksum    = 0xa356
     src       = 0.0.0.0
     dst       = 255.255.255.255
     \options   \
###[ UDP ]### 
        sport     = bootpc
        dport     = bootps
        len       = 308
        chksum    = 0x112a
###[ BOOTP ]### 
          

In [53]:
local_ip = "192.168.100.2"
internet_ip = "192.168.200.2"
network_prefix = "192.168.100"

Write a function `parse_ethernet_frame` that takes a single parameter `frame`. This function should parse an Ethernet frame and return its components. Follow these steps:

1. Extract the destination MAC address:
   - Use the first 6 bytes of the frame
   - Convert it to a hexadecimal string with colons between each byte

2. Extract the source MAC address:
   - Use the next 6 bytes of the frame (bytes 6-12)
   - Convert it to a hexadecimal string with colons between each byte

3. Extract the EtherType:
   - Use the next 2 bytes of the frame (bytes 12-14)
   - Unpack these bytes as a big-endian unsigned short integer

4. Extract the payload:
   - Use all remaining bytes of the frame (from byte 14 onwards)

5. Return the extracted components in the following order:
   destination MAC, source MAC, EtherType, and payload

Notes:
- Use the `bytes.hex()` method with the `':'` separator for MAC addresses
- Use `struct.unpack()` with the format string `'!H'` for the EtherType

In [74]:
def parse_ethernet_frame(frame):
    dst_mac = frame[:6].hex(':')
    src_mac = frame[6:12].hex(':')
    ethertype = struct.unpack('!H', frame[12:14])[0]
    payload = frame[14:]
    return dst_mac, src_mac, ethertype, payload

Write a function `parse_ip_header` that takes a single parameter `ip_data`. This function should parse an IP header and return its components. Follow these steps:

1. Unpack the first 20 bytes of the IP header:
   - Use `struct.unpack()` with the format string `'!BBHHHBBH4s4s'`
   - Store the result in a variable named `iph`

2. Extract the version and IHL (Internet Header Length):
   - The first byte contains both version and IHL
   - Extract version by right-shifting the first byte by 4 bits
   - Extract IHL by masking the first byte with 0xF and multiplying by 4

3. Extract the TTL (Time To Live):
   - This is the 6th byte in the header (index 5 in `iph`)

4. Extract the protocol:
   - This is the 7th byte in the header (index 6 in `iph`)

5. Extract and convert source IP address:
   - Use the 9th 4-byte chunk from `iph` (index 8)
   - Convert from network byte order to a string using `socket.inet_ntoa()`

6. Extract and convert destination IP address:
   - Use the 10th 4-byte chunk from `iph` (index 9)
   - Convert from network byte order to a string using `socket.inet_ntoa()`

7. Extract the payload:
   - This is all data after the header (use `ip_data[ihl:]`)

8. Return all extracted components in the following order:
   version, IHL, TTL, protocol, source IP, destination IP, and payload

Notes:
- Import necessary modules (you'll need `struct` and `socket`)
- The `inet_ntoa()` function converts an IP address from 32-bit packed binary format to a string

In [75]:
def parse_ip_header(ip_data):
    iph = struct.unpack('!BBHHHBBH4s4s', ip_data[:20])
    version_ihl = iph[0]
    version = version_ihl >> 4
    ihl = (version_ihl & 0xF) * 4
    ttl = iph[5]
    protocol = iph[6]
    src_ip = socket.inet_ntoa(iph[8])
    dst_ip = socket.inet_ntoa(iph[9])
    return version, ihl, ttl, protocol, src_ip, dst_ip, ip_data[ihl:]

In [76]:
def parse_tcp_udp_header(data, protocol):
    if protocol in [6, 17]:  # TCP or UDP
        ports = struct.unpack('!HH', data[:4])
        src_port, dst_port = ports
        return src_port, dst_port
    return None, None

# IP Header Creation

Given a source and destination IP, create an IP header. The IP header is some data at the start of the packet that contains routing information. It's formatted as per the IP standard (https://en.wikipedia.org/wiki/IPv4).

## Header Structure

1. **Version (4 bits)**: 4 for IPv4
2. **Header Length (4 bits)**: Fixed length of 5 (no options)
3. **Type of Service (1 byte)**:
   - Set to 0 (default)
   - Options can be found at https://en.wikipedia.org/wiki/Differentiated_services
   - Includes Explicit Congestion Notification (also set to 0)
4. **Total Length (2 bytes)**: Length of the entire packet (header + data)
5. **Identification (2 bytes)**: Set to a random number
6. **Fragmentation (2 bytes)**: Not used in this case
7. **Time To Live (TTL) (1 byte)**: Decremented each time the packet is forwarded
8. **Protocol (1 byte)**:
   - Set to 6 for TCP
   - Full list: https://en.wikipedia.org/wiki/List_of_IP_protocol_numbers
9. **Header Checksum (2 bytes)**: Set to 0 for now, to be calculated based on the header

## Question

1. What is the maximum possible length of data that can be sent in a single packet, excluding the header, based on what we know so far?

In [77]:
def create_ip_header(src_ip, dst_ip, protocol, total_length, ttl):
    ip_ihl_ver = (4 << 4) + 5
    ip_dscp_ecn = 0
    ip_id = random.randint(0, 65535)
    ip_frag_off = 0
    ip_checksum = 0
    src_addr = socket.inet_aton(src_ip)
    dst_addr = socket.inet_aton(dst_ip)
    
    ip_header = struct.pack('!BBHHHBBH4s4s',
        ip_ihl_ver, ip_dscp_ecn, total_length,
        ip_id, ip_frag_off, ttl, protocol, ip_checksum,
        src_addr, dst_addr)
    
    return ip_header

Implement a function `calculate_checksum(data)` that computes the Internet Checksum for the given packet.

> The checksum field is the 16 bit one's complement of the one's complement sum of all 16 bit words in the header. For purposes of computing the checksum, the value of the checksum field is zero.

You can read more about it on [Wikipedia](https://en.wikipedia.org/wiki/Internet_checksum)

This checksum is commonly used in network protocols like IP, TCP, and UDP. Follow these general steps:

1. Prepare the data:
   - Ensure the data has an even number of bytes, padding if necessary.

2. Divide the data into 16-bit words:
   - Convert the byte sequence into a series of 16-bit integers.

3. Sum the 16-bit words:
   - Add all the 16-bit words together.

4. Handle overflow:
   - If there's any overflow beyond 16 bits during addition, add the overflow back to the sum.

5. Compute the one's complement:
   - Take the one's complement of the final sum.

6. Return the result:
   - The checksum should be a 16-bit value.

Questions:
What should the checksum of a packet that was transmitted correctly be?

In [78]:
def calculate_checksum(data):
    if len(data) % 2 == 1:
        data += b'\0'
    words = struct.unpack('!%dH' % (len(data) // 2), data)
    return (~sum(words) & 0xFFFF)

In [79]:
# Routing table to keep track of which interface to use for a given destination
routing_table = {}

def update_routing_table(mac_address, interface):
    routing_table[mac_address] = interface

Implement a function `handle_nat` that performs Network Address Translation (NAT) for outgoing packets. The function should take the following parameters:

1. `protocol`: The protocl of the packet (e.g., 'TCP', 'UDP')
2. `src_ip`: Original source IP address
3. `src_port`: Original source port
4. `dst_ip`: Destination IP address
5. `dst_port`: Destination port

The function should perform the following steps:

1. Connection Identification:
   - Create a unique identifier for the connection using the protocol, source IP, and source port.
   - Log or print the NAT operation details for debugging purposes.

2. NAT Table Lookup:
   - Check if the connection identifier exists in the NAT table.

3. New Connection Handling:
   - If the connection is new (not in the NAT table):
     a. Generate a new 'public' port number from the dynamic/private port range (49152-65535).
     b. Create a new entry in the NAT table mapping the connection ID to the new public port.
     c. Create a reverse mapping in the NAT table for incoming packets.

4. Return Translation:
   - Return the public IP address (internet-facing IP) and the translated port number.

Notes:
- The function should handle potential port conflicts and ensure unique mappings.

In [80]:
# NAT table to keep track of connections

nat_table = {}

def handle_nat(protocol, src_ip, src_port, dst_ip, dst_port):
    # Generate a unique identifier for this connection
    conn_id = f"{protocol}:{src_ip}:{src_port}"
    print(f"NAT called: {conn_id} -> {dst_ip}:{dst_port}")

    if conn_id not in nat_table:
        # Assign a new 'public' port for this connection
        public_port = random.randint(49152, 65535)
        nat_table[conn_id] = public_port
        nat_table[f"{protocol}:{internet_ip}:{public_port}"] = (src_ip, src_port)
    
    return internet_ip, nat_table[conn_id]

In [81]:
def forward_frame(frame, output_interface):
    sendp(Raw(frame), iface=output_interface, verbose=False)
    print(f"Forwarded frame via {output_interface}")

In [82]:
def modify_ip_header(ip_header, new_src_ip, new_dst_ip, new_ttl):
    new_ip_header = create_ip_header(new_src_ip, new_dst_ip, ip_header[6], len(ip_header), new_ttl)
    checksum = calculate_checksum(new_ip_header)
    return new_ip_header[:10] + struct.pack('!H', checksum) + new_ip_header[12:]

In [83]:
def modify_tcp_udp_header(header, new_src_port=None, new_dst_port=None):
    src_port, dst_port = struct.unpack('!HH', header[:4])
    if new_src_port is not None:
        src_port = new_src_port
    if new_dst_port is not None:
        dst_port = new_dst_port
    return struct.pack('!HH', src_port, dst_port) + header[4:]

Implement a function `create_ethernet_frame` that constructs an Ethernet frame from its components. The function should take four parameters:

1. `dst_mac`: The destination MAC address
2. `src_mac`: The source MAC address
3. `ethertype`: The EtherType field
4. `payload`: The frame's payload data

This is just the reverse of `parse_etehrnet_fame`; you may want to refer back to that.

In [84]:
def create_ethernet_frame(dst_mac, src_mac, ethertype, payload):
    return struct.pack('!6s6sH', bytes.fromhex(dst_mac.replace(':', '')),
                                 bytes.fromhex(src_mac.replace(':', '')),
                                 ethertype) + payload

Implement a function `handle_outgoing_packet` that processes an outgoing network packet. The function should take the following parameters:

1. `src_ip`: Source IP address
2. `src_port`: Source port
3. `dst_ip`: Destination IP address
4. `dst_port`: Destination port
5. `protocol`: Network protocol (e.g., TCP, UDP)
6. `ip_header`: The original IP header
7. `ip_payload`: The original IP payload (including transport layer header and data)

The function should perform the following steps:

1. Apply NAT to potentially modify the source IP and port.

2. Decrement the TTL value from the IP header.

3. Update the IP header with the new source IP (post-NAT) and TTL.

4. Update the transport layer header (TCP or UDP) with the new source port (post-NAT).

5. Combine the modified IP header and transport layer header.

6. Determine the destination MAC address using a routing table.
   - If not found, use a broadcast MAC address.

7. Get the MAC address of the outgoing network interface.

8. Construct an Ethernet frame using the resolved MAC addresses, appropriate EtherType for IP (0x0800), and the modified payload.

9. Return the created Ethernet frame and the identifier for the outgoing network interface.

In [85]:
def handle_outgoing_packet(src_ip, src_port, dst_ip, dst_port, protocol, ip_header, ip_payload):
    new_src_ip, new_src_port = handle_nat(protocol, src_ip, src_port, dst_ip, dst_port)
    new_ttl = ip_header[5] - 1
    new_ip_header = modify_ip_header(ip_header, new_src_ip, dst_ip, new_ttl)
    new_transport_header = modify_tcp_udp_header(ip_payload, new_src_port=new_src_port)
    new_payload = new_ip_header + new_transport_header
    dst_mac = routing_table.get(dst_ip, "ff:ff:ff:ff:ff:ff")
    src_mac = get_if_hwaddr(internet_interface)
    return create_ethernet_frame(dst_mac, src_mac, 0x0800, new_payload), internet_interface

In [86]:
def handle_incoming_packet(src_ip, dst_ip, dst_port, protocol, ip_header, ip_payload):
    conn_id = f"{protocol}:{internet_ip}:{dst_port}"
    if conn_id in nat_table:
        original_dst_ip, original_dst_port = nat_table[conn_id]
        new_ttl = ip_header[5] - 1
        new_ip_header = modify_ip_header(ip_header, src_ip, original_dst_ip, new_ttl)
        new_transport_header = modify_tcp_udp_header(ip_payload, new_dst_port=original_dst_port)
        new_payload = new_ip_header + new_transport_header
        dst_mac = routing_table.get(original_dst_ip, "ff:ff:ff:ff:ff:ff")
        src_mac = get_if_hwaddr(local_interface)
        return create_ethernet_frame(dst_mac, src_mac, 0x0800, new_payload), local_interface
    else:
        print(f"Dropping packet: No NAT entry for {dst_ip}:{dst_port}")
        return None, None

In [87]:

def packet_handler(packet):
    frame = bytes(packet)
    dst_mac, src_mac, ethertype, payload = parse_ethernet_frame(frame)

    print(f"Received frame: {src_mac} -> {dst_mac}")

    if ethertype != 0x0800:  # Not IPv4
        print(f"Non-IPv4 frame received. EtherType: {hex(ethertype)}")
        return

    version, ihl, ttl, protocol, src_ip, dst_ip, ip_payload = parse_ip_header(payload)
    src_port, dst_port = parse_tcp_udp_header(ip_payload, protocol)

    print(f"IP packet: {src_ip}:{src_port} -> {dst_ip}:{dst_port}")

    update_routing_table(src_mac, packet.sniffed_on)

    if ttl <= 1:
        print(f"Dropping packet: {src_ip} -> {dst_ip} due to TTL expiration")
        return

    if src_ip.startswith(network_prefix):  # Outgoing packet from local network
        new_frame, output_interface = handle_outgoing_packet(src_ip, src_port, dst_ip, dst_port, protocol, payload[:20], ip_payload)
    
    elif dst_ip == internet_ip:  # Incoming packet from internet
        new_frame, output_interface = handle_incoming_packet(src_ip, dst_ip, dst_port, protocol, payload[:20], ip_payload)

    else:
        print(f"Dropping packet: {src_ip} -> {dst_ip} because it is not destined for this router")
        return

    if new_frame and output_interface:
        forward_frame(new_frame, output_interface)

In [88]:
def start_router():
    print("Low-level Ethernet router started. Listening for frames...")
    sniff(iface=[internet_interface, local_interface], prn=packet_handler, store=0)

In [89]:
start_router()

Low-level Ethernet router started. Listening for frames...
Received frame: d4:57:63:c6:e8:17 -> 01:00:5e:00:00:fb
IP packet: 10.11.2.22:5353 -> 224.0.0.251:5353
Dropping packet: 10.11.2.22 -> 224.0.0.251 because it is not destined for this router
Received frame: d4:57:63:c6:e8:17 -> 01:00:5e:00:00:fb
IP packet: 10.11.2.22:5353 -> 224.0.0.251:5353
Dropping packet: 10.11.2.22 -> 224.0.0.251 because it is not destined for this router
Received frame: d4:57:63:c6:e8:17 -> 33:33:00:00:00:fb
Non-IPv4 frame received. EtherType: 0x86dd
Received frame: d4:57:63:c6:e8:17 -> 33:33:00:00:00:fb
Non-IPv4 frame received. EtherType: 0x86dd
Received frame: d4:57:63:c6:e8:17 -> 33:33:00:00:00:fb
Non-IPv4 frame received. EtherType: 0x86dd
Received frame: d4:57:63:c6:e8:17 -> 33:33:00:00:00:fb
Non-IPv4 frame received. EtherType: 0x86dd
Received frame: f8:ff:c2:2b:ec:9b -> ff:ff:ff:ff:ff:ff
IP packet: 10.11.0.16:62653 -> 10.11.3.255:21027
Dropping packet: 10.11.0.16 -> 10.11.3.255 because it is not destined

Challenges:
1. Block traffic to a blacklist of IP addresses.
2. Maliciously re-route traffic from google.com to https://elgoog.im/